# Left, inner, outer, cross and fast-track joins
By the end of this lecture you will be able to:
- do left, inner, outer and cross joins between `DataFrames`
- validate joins
- do fast-track joins on sorted integer columns

In [ ]:
import polars as pl
pl.Config.set_tbl_rows(10)

## CITES Dataset
For this lecture we use an extract from a database on international trade in endangered species gathered by the CITES organisation.

This CSV has an extract of CITES trade data for 2021

In [ ]:
csv_file = "../data/cites_extract.csv"

In [ ]:
df_CITES = pl.read_csv(csv_file)
df_CITES

The `DataFrame` shows:
- the `Year` in which the trade occured
- the `Importer` and `Exporter` country in 2-digit ISO country codes
- the scientific name for the `Taxon` and
- the `Quantity` of items in the trade

For importers and exporters we would like to have:
- the full country name
- the region of that country

We will join the trade data to the ISO country metadata in the following CSV

In [ ]:
iso_csv_file = "../data/countries_extract.csv"

In [ ]:
df_ISO = pl.read_csv(iso_csv_file)
df_ISO

This `DataFrame` has:
- `alpha-2`: the 2-letter country code
- `name`: the full name of the country
- `region`: the region of the country

## Left join
In a left join we go through the left `DataFrame` row-by-row and try to add the data from the right `DataFrame` based on a join column in each `DataFrame`

In [ ]:
(
    df_CITES
    .join(
        df_ISO,
        left_on="Importer",
        right_on="alpha-2", 
        how="left")
)

We join on:
- the `Importer` column for the left `DataFrame` and 
- `alpha-2` for the right `DataFrame`

In this case:
- we now have the `name` and `region` columns from `dfISO` that give the name and region for the importing country
- we have `null` in the last row because the country cide "UA" is missing from `dfISO`.

We want to rename `name` and `region` columns to reflect that these are the importer values

In [ ]:
(
    df_CITES
    .join(
     df_ISO,
     left_on="Importer",
     right_on="alpha-2", 
     how="left"
   )
   .rename(
     {
         "name":"name_importer",
         "region":"region_importer"
     }
  )
)

We will join the exporter values in the exercises.

The `join` method also has a `suffix` option. This adds a suffix to the column names in the right `DataFrame` *for column names that occur in both `DataFrames`*.

## Inner join
In an inner join we only retain the rows in the left `DataFrame` where we can join to a value in the right `DataFrame`


In [ ]:
(
    df_CITES
    .join(
        df_ISO,
        left_on="Importer",
        right_on="alpha-2", 
        how="inner"
    )
)

The final row that had `null` values for `name` and `region` is not present with an inner join.

## Outer join
In an outer join we return all rows from both `DataFrames` with `null` values where the value in the join column is not present in both `DataFrames`

In [ ]:
(
    df_CITES
    .join(
        df_ISO,
        left_on="Importer",
        right_on="alpha-2", 
        how="outer"
    )
)

In the first row there are `null` values for the `df_CITES` columns as `BJ` is present in `dfISO` but not in `df_CITES`

## Cross join
In a cross join we create rows with all the combinations of the values in the join columns

In [ ]:
(
    df_CITES
    .join(
        df_ISO,
        left_on="Importer",
        right_on="alpha-2", 
        how="cross"
    )
)

## Validating joins
It is easy to unintentionally lose data when joining large `DataFrames`. Polars allows you to validate your joins to ensure this isn't happening.

In the example below we create a `DataFrame` that records purchases from suppliers in `purchases_df`. We also create a `DataFrame` that identifies suppliers in `supplier_df`. This `DataFrame` has an error because there are two entries for `supplier_id=1`

When we join the supplier info to the purchases we expect to get a single mapping from `supplier_id` to a row in `purchases_df`. However, we get an extra row from the two entries for `supplier_id=1`

In [ ]:
purchases_df = pl.DataFrame(
    {'supplier_id':[0,1],'quantity':[100,200]}
)
supplier_df = pl.DataFrame(
    {'supplier_id':[0,1,1],'country':['UK','France','Germany']}
)
purchases_df.join(supplier_df,on='supplier_id',how='left')

To detect this happening we can pass the `validate='1:1'` argument. This throws an `Exception` if the join on `supplier_id` is not unique

In [ ]:
(
    purchases_df
    .join(
        supplier_df,on='supplier_id',how='left',validate='1:1')
)

In this case we may not actually be concerned if the `supplier_id` is repeated in `purchases_df` as this could reflect multiple purchases from the same supplier. But we do want a single entry for each `supplier_id` in `supplier_df`.

Instead we would use the many-to-one (`m:1`) validation check to check that the `supplier_id` is unique in the right `DataFrame`. Again this produces an `Exception` as we repeated `supplier_ids` in `supplier_df`

In [ ]:
(
    purchases_df
    .join(
        supplier_df,on='supplier_id',how='left',validate='m:1')
)

Similarly, if we wanted to have no repeated join keys in the left `DataFrame` we would use the one-to-many `1:m` validation.

In summary, if we want unique keys in:
- both `DataFrames` use `1:1`
- the left `DataFrame` only use `1:m`
- the right `DataFrame` only use `m:1`

## Joins in lazy mode
We can do joins in lazy mode by joining on `LazyFrames` instead of `DataFrames`

Join operations - and cross joins in particular - can be memory intensive. Evaluating large queries with streaming can reduce the memory pressure by running the operation in batches.

In this example we do the `join` in streaming mode by:
- converting `df_CITES` and `dfISO` to `LazyFrames` before joining
- calling `collect(streaming=True)` at the end to evaluate in streaming mode

In [ ]:
(
    df_CITES
    .lazy()
    .join(
        df_ISO.lazy(),
        left_on="Importer",
        right_on="alpha-2", 
        how="cross"
    )
    .collect(streaming=True)
)

## Exercises

In the exercises you will develop your understanding of:
- doing a left join of two `DataFrames`
- doing an inner join of two `DataFrames`
- doing fast-track joins on sorted integer columns

### Exercise 1
Do a left join of the CITES trade extract with the country data on the importer column

In [ ]:
df_CITES = pl.read_csv(csv_file)
df_ISO = pl.read_csv(iso_csv_file)
(
    <blank>
)

Now add a left join with the country data on the **exporter** column. Validate that the join keys are unique in the right `DataFrame` with country data

In [ ]:
(
    <blank>
)

Do an inner join with the country data for both importer and exporter

In [ ]:
(
    <blank>
)

### Exercise 2
In this exercise we see the effect of joins on sorted integers

We first create a pre-sorted array of `N` integers to be the join keys.

We control the `cardinality` - the number of unique join keys - with the `cardinality` variable

In [ ]:
import numpy as np
np.random.seed(0)

N = 100_000
cardinality = N // 2

We create a left-hand `DataFrame` with:
- a sorted `id` column and
- a random `values` column

We create a right-hand `DataFrame` with
- a sorted `id` column
- a metadata column (equal to the `id` column in this case)

In [ ]:
def createDataFrames(N:int,cardinality:int):
    # Create a random array with values up to cardinality and then sort it to be the `id` column
    sortedArray = np.sort(np.random.randint(0,cardinality,N))
    df_left = (
        pl.DataFrame(
            {
                "id":[i for i in sortedArray],
                "values":np.random.standard_normal(N)
            }
        )
    )
    # We create the right-hand `DataFrame` with the `id` column and arbitrary metadata 
    df_right = (
    pl.DataFrame(
        {
            "id":[i for i in range(cardinality)],
            "meta":[i for i in range(cardinality)]
        }
    )
)
    return df_left, df_right
df_left,df_right = createDataFrames(N = N, cardinality = cardinality)
df_left.head()

In [ ]:
df_right.head()

Check the flags if Polars knows the `id` column is sorted on the left and right `DataFrames`

In [ ]:
print(<blank>)
print(<blank>)

Time the performance for an unsorted join

In [ ]:
%%timeit -n1 -r3
(
  <blank>  
)

Create new `DataFrames` and tell Polars that the `id` columns are sorted

In [ ]:
df_left_sorted = (
    <blank>
)
                
df_right_sorted = (
    <blank>
)


Check the flags to see if Polars knows the `id` column is sorted on these new `DataFrames`

In [ ]:
print(<blank>)
print(<blank>)

Time the sorted join performance

In [ ]:
%%timeit -n1 -r3
(
  <blank>  
)

Compare performance if only the left `DataFrame` is sorted. Hint: use `df_left_sorted` and `df_right`

In [ ]:
%%timeit -n1 -r3
(
  <blank>  
)

Compare the relative performance between sorted and unsorted joins when `cardinality` is low (say `cardinality = 100`)

## Solutions

### Solution to Exercise 1
Do a left join of the CITES trade extract with the country data on the importer column

In [ ]:
df_CITES = pl.read_csv(csv_file)
df_ISO = pl.read_csv(iso_csv_file)
(
    df_CITES
    .join(
        df_ISO,
        left_on="Importer",
        right_on="alpha-2", 
        how="left"
    )
    .rename(
        {"name":"name_importer","region":"region_importer"}
    )
)

Now add a left join with the country data on the **exporter** column. Validate that the join keys are unique in the right `DataFrame` called `dfISO`

In [ ]:
(
    df_CITES
    .join(
        df_ISO,
        left_on="Importer",
        right_on="alpha-2", 
        how="left"
    )
    .rename(
        {"name":"name_importer","region":"region_importer"}
    )
    .join(
        df_ISO,
        left_on="Exporter",
        right_on="alpha-2", 
        how="left",
        validate="m:1"
    )
    .rename({"name":"name_exporter","region":"region_exporter"})
)

Do an inner join with the country data for both importer and exporter

In [ ]:
(
    df_CITES
    .join(
        df_ISO,
        left_on="Importer",
        right_on="alpha-2", 
        how="inner"
    )
    .rename(
        {"name":"name_importer","region":"region_importer"}
    )
    .join(
        df_ISO,
        left_on="Exporter",
        right_on="alpha-2", 
        how="inner"
    )
    .rename(
        {"name":"name_exporter","region":"region_exporter"}
    )
)

### Solution to Exercise 2

In [ ]:
import numpy as np
np.random.seed(0)
N = 10_000_000
cardinality = N // 2
df_left,df_right = createDataFrames(N = N, cardinality = cardinality)

Check the flags to see if Polars knows the `id` column is sorted on the left and right `DataFrames`

In [ ]:
print(df_left["id"].flags)
print(df_right["id"].flags)

Time the performance for an unsorted join

In [ ]:
%%timeit -n1 -r3
(
    df_left.join(df_right,on="id")
)

Create new `DataFrames` and tell Polars that the `id` columns are sorted

In [ ]:
df_left_sorted = (
    df_left
    .with_columns(pl.col("id").set_sorted())
)
                
df_right_sorted = (
    df_right
    .with_columns(pl.col("id").set_sorted())
)


Check to see if Polars knows the `id` columns are sorted

In [ ]:
print(df_left_sorted["id"].flags)
print(df_right_sorted["id"].flags)

Time the sorted join performance

In [ ]:
%%timeit -n1 -r3

(
    df_left_sorted.join(df_right_sorted,left_on="id",right_on="id")
)

This is much faster than the unsorted joins

Compare performance if only the left `DataFrame` is sorted

In [ ]:
%%timeit -n1 -r3
(
    df_left_sorted.join(df_right,left_on="id",right_on="id")
)

There is still a benefit if just the left `DataFrame` is sorted

In [ ]:
%%timeit -n1 -r3
(
    df_left.join(df_right_sorted,left_on="id",right_on="id")
)

So there is no performance benefit from just the right `DataFrame` being sorted 

Compare the relative performance when `cardinality` is low (say `cardinality = 100`)

With low cardinality the overall joins are much faster but the differences in performances from sorting are much smaller